# 🔍 Background Subtraction Object Detection v4.1

ตรวจจับวัตถุโดยเปรียบเทียบกับ Background และ **ใช้ค่าจากกล้องควบคุม Gripper**

**🆕 v4.1 Changes:**
- ลบ SERVO_OPEN/CLOSE คงที่
- ใช้ค่าความกว้างจากกล้องควบคุม gripper
- เปิด/คีบ/ปล่อย ตามขนาดวัตถุจริง

**วิธีการ:**
1. ถ่าย Background (พื้นเปล่า)
2. วางวัตถุ → ระบบตรวจจับความแตกต่าง
3. วัดขนาดวัตถุ (pixels → mm)
4. Gripper เปิด/คีบตามขนาดจากกล้อง
5. หยิบวัตถุ

## 1️⃣ Imports & Setup

In [ ]:
!pip install pyserial opencv-python numpy matplotlib

In [1]:
import cv2
import numpy as np
import time
import socket
import serial

print("✓ Imports loaded")

✓ Imports loaded


## 2️⃣ Configuration

In [2]:
# Robot IP
ROBOT_IP = '192.168.1.6'

# ESP32 Serial Port
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200

# Camera ID
CAMERA_ID = 2

# Homography Matrix (from calibrate.py)
HOMOGRAPHY_MATRIX = np.array([
    [0.005703976266962427, -0.3265299161278153, 88.58634169557483],
    [-0.47704058225560797, 0.015355046930804153, 172.0941543570439],
    [-0.00029949919510557677, 0.00018728182448344945, 1.0],
], dtype=np.float32)

# =====================================================
# CALIBRATION: Pixels per mm
# ⚠️ ต้อง Calibrate ค่านี้ด้วยวัตถุที่ทราบขนาด!
# =====================================================
PIXELS_PER_MM = 3.0  # ค่าเริ่มต้น (ต้องปรับ)

# Z Heights
Z_FLOOR = -64     # หัวคีบติดพื้น
Z_SAFE = -50      # ความสูงปลอดภัย
R_ANGLE = 59.21

# Drop position
DROP_POS = (-27.61, 74.19, 171.88, 0)

# =====================================================
# 🆕 Gripper Configuration (Dynamic from Camera)
# =====================================================
# Servo angle to gripper width mapping:
# - 22° = 74mm (fully open)
# - 96° = 5mm (fully closed)
GRIPPER_SERVO_OPEN_ANGLE = 22   # มุมเปิดสุด
GRIPPER_SERVO_CLOSE_ANGLE = 96  # มุมปิดสุด
GRIPPER_MAX_WIDTH_MM = 74       # ความกว้างที่มุม 22°
GRIPPER_MIN_WIDTH_MM = 5        # ความกว้างที่มุม 96°

# Margin settings
GRIPPER_OPEN_MARGIN_MM = 15     # เปิดกว้างกว่าวัตถุกี่ mm ก่อนหยิบ
GRIPPER_GRIP_MARGIN_MM = 3      # คีบแน่นกว่าวัตถุกี่ mm

# Background subtraction settings
MIN_OBJECT_AREA = 1000  # Minimum area to detect (pixels²)
THRESHOLD_VALUE = 30    # Threshold for difference

print("✓ Configuration loaded")
print(f"  PIXELS_PER_MM: {PIXELS_PER_MM}")
print(f"  Gripper range: {GRIPPER_MIN_WIDTH_MM}-{GRIPPER_MAX_WIDTH_MM} mm")
print(f"  Open margin: +{GRIPPER_OPEN_MARGIN_MM} mm")
print(f"  Grip margin: -{GRIPPER_GRIP_MARGIN_MM} mm")

✓ Configuration loaded
  PIXELS_PER_MM: 3.0
  Gripper range: 5-74 mm
  Open margin: +15 mm
  Grip margin: -3 mm


## 3️⃣ GripperController Class (Dynamic Width)

In [3]:
class GripperController:
    """🆕 Gripper controller using camera measurements for dynamic width"""
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.current_angle = GRIPPER_SERVO_OPEN_ANGLE
        self.target_object_width = None  # เก็บขนาดวัตถุที่จะหยิบ
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper connected on {self.port}")
            return True
        except Exception as e:
            print(f"❌ Connection failed: {e}")
            return False
    
    def disconnect(self):
        if self.serial:
            self.serial.close()
            print("Disconnected")
    
    def send_command(self, cmd):
        if not self.serial:
            return None
        self.serial.reset_input_buffer()
        self.serial.write((cmd + '\n').encode())
        time.sleep(0.3)
        response = ""
        while self.serial.in_waiting:
            response += self.serial.readline().decode().strip() + "\n"
        return response.strip()
    
    def mm_to_angle(self, width_mm):
        """
        แปลงความกว้าง (mm) เป็นมุม Servo
        Linear mapping: 74mm → 22°, 5mm → 96°
        """
        width = max(GRIPPER_MIN_WIDTH_MM, min(GRIPPER_MAX_WIDTH_MM, width_mm))
        angle_range = GRIPPER_SERVO_CLOSE_ANGLE - GRIPPER_SERVO_OPEN_ANGLE  # 74
        width_range = GRIPPER_MAX_WIDTH_MM - GRIPPER_MIN_WIDTH_MM  # 69
        angle = GRIPPER_SERVO_CLOSE_ANGLE - ((width - GRIPPER_MIN_WIDTH_MM) / width_range) * angle_range
        return int(angle)
    
    def open_for_object(self, object_width_mm):
        """
        🆕 เปิด gripper กว้างพอสำหรับวัตถุ (วัตถุ + margin)
        ใช้แทน open() คงที่
        """
        self.target_object_width = object_width_mm
        open_width = object_width_mm + GRIPPER_OPEN_MARGIN_MM
        open_width = min(GRIPPER_MAX_WIDTH_MM, open_width)  # ไม่เกิน max
        angle = self.mm_to_angle(open_width)
        print(f"🦾 Opening for {object_width_mm:.1f}mm object → open to {open_width:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
        self.current_angle = angle
    
    def grip_object(self, object_width_mm):
        """
        🆕 คีบวัตถุให้แน่นพอดี (ขนาดวัตถุ - margin เล็กน้อย)
        ใช้ค่าจากกล้อง
        """
        # คีบแน่นกว่าวัตถุเล็กน้อย
        grip_width = max(GRIPPER_MIN_WIDTH_MM, object_width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_width)
        # เพิ่มมุมอีกเล็กน้อยเพื่อให้คีบแน่น (+3°)
        angle = min(GRIPPER_SERVO_CLOSE_ANGLE, angle + 3)
        print(f"🦾 Gripping {object_width_mm:.1f}mm object → close to {grip_width:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
        self.current_angle = angle
    
    def release(self):
        """
        🆕 ปล่อยวัตถุ - เปิดกว้างพอสำหรับวัตถุที่ถืออยู่
        """
        if self.target_object_width:
            release_width = self.target_object_width + 10
        else:
            release_width = GRIPPER_MAX_WIDTH_MM  # เปิดสุด
        release_width = min(GRIPPER_MAX_WIDTH_MM, release_width)
        angle = self.mm_to_angle(release_width)
        print(f"🦾 Releasing → open to {release_width:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
        self.current_angle = angle
        self.target_object_width = None
    
    def calculate_grip_angle(self, object_width_mm):
        """
        (สำหรับแสดงผลเท่านั้น) คำนวณมุมที่จะใช้คีบ
        """
        grip_width = max(GRIPPER_MIN_WIDTH_MM, object_width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_width)
        return min(GRIPPER_SERVO_CLOSE_ANGLE, angle + 3)

print("✓ GripperController class loaded (dynamic width from camera)")

✓ GripperController class loaded (dynamic width from camera)


## 4️⃣ DobotControllerTCP Class

In [4]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None):
        self.dashboard_port = 29999
        self.sock_dashboard = None
        self.homography_matrix = homography_matrix
        
    def connect(self, ip):
        try:
            self.sock_dashboard = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock_dashboard.settimeout(5)
            self.sock_dashboard.connect((ip, self.dashboard_port))
            
            print("Clearing Errors...")
            self.send_command("ClearError()")
            time.sleep(0.5)
            
            print("Enabling Robot...")
            self.send_command("EnableRobot()")
            time.sleep(4)
            
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Connection Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock_dashboard:
            try:
                self.sock_dashboard.send((cmd + "\n").encode("utf-8"))
                return self.sock_dashboard.recv(1024).decode("utf-8")
            except Exception as e:
                print(f"Command Error: {e}")
        return None

    def home(self):
        print("🤖 Moving to HOME...")
        self.send_command("JointMovJ(0.19,-12.21,38.29,59.21)")
        time.sleep(4)
        print("✅ HOME")

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   Sending: {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait_time=3):
        self.move_to(x, y, z, r)
        time.sleep(wait_time)

    def pixel_to_robot(self, u, v):
        if self.homography_matrix is None:
            return None, None
        point = np.array([u, v, 1], dtype=np.float32)
        result = np.dot(self.homography_matrix, point)
        return result[0] / result[2], result[1] / result[2]

print("✓ DobotControllerTCP class loaded")

✓ DobotControllerTCP class loaded


## 5️⃣ Object Detection Functions

In [5]:
class ObjectDetector:
    def __init__(self):
        self.background = None
        self.background_gray = None
        
    def set_background(self, frame):
        """ตั้งค่า Background"""
        self.background = frame.copy()
        self.background_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        self.background_gray = cv2.GaussianBlur(self.background_gray, (5, 5), 0)
        print("✅ Background captured!")
    
    def detect_objects(self, frame, threshold=40, min_area=2000):
        """
        ตรวจจับวัตถุโดยเปรียบเทียบกับ Background
        """
        if self.background_gray is None:
            return []
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (5, 5), 0)
        
        diff = cv2.absdiff(self.background_gray, gray)
        _, thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        thresh = cv2.erode(thresh, kernel, iterations=2)
        thresh = cv2.dilate(thresh, kernel, iterations=1)
        thresh = cv2.erode(thresh, kernel, iterations=1)
        
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        objects = []
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > min_area:
                hull = cv2.convexHull(contour)
                x, y, w, h = cv2.boundingRect(hull)
                cx = x + w // 2
                cy = y + h // 2
                objects.append((x, y, w, h, cx, cy, hull))
        
        objects.sort(key=lambda o: o[2] * o[3], reverse=True)
        return objects
    
    def measure_object(self, obj, pixels_per_mm):
        """วัดขนาดวัตถุ"""
        x, y, w, h, cx, cy, contour = obj
        
        width_mm = w / pixels_per_mm
        height_mm = h / pixels_per_mm
        
        rect = cv2.minAreaRect(contour)
        rect_w, rect_h = rect[1]
        grip_width_px = min(rect_w, rect_h)
        grip_width_mm = grip_width_px / pixels_per_mm
        
        return width_mm, height_mm, grip_width_mm

print("✓ ObjectDetector class loaded")

✓ ObjectDetector class loaded


## 6️⃣ Connect Devices

In [7]:
# Connect Gripper
gripper = GripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
gripper.connect()

✅ Gripper connected on COM9


True

In [8]:
# =================================================================
# 🧪 Test Gripper Dynamic Control
# =================================================================
# ทดสอบการกางหุบตามความกว้างที่ระบุ (มม.)

print("=" * 50)
print("🧪 Gripper Dynamic Test")
print("=" * 50)
print("Commands:")
print("  [number] = Set gripper width (mm), e.g. '30' = 30mm")
print("  o = Open fully (74mm)")  
print("  c = Close fully (5mm)")
print("  t = Test sequence (open 50mm → grip 30mm → release)")
print("  q = Quit")
print("=" * 50)

while True:
    cmd = input("\n🦾 Enter command: ").strip().lower()
    
    if cmd == 'q':
        print("Exit test mode")
        break
    
    elif cmd == 'o':
        # Open fully
        angle = gripper.mm_to_angle(GRIPPER_MAX_WIDTH_MM)
        print(f"Opening to {GRIPPER_MAX_WIDTH_MM}mm ({angle}°)")
        gripper.send_command(f'G{angle}')
        
    elif cmd == 'c':
        # Close fully
        angle = gripper.mm_to_angle(GRIPPER_MIN_WIDTH_MM)
        print(f"Closing to {GRIPPER_MIN_WIDTH_MM}mm ({angle}°)")
        gripper.send_command(f'G{angle}')
        
    elif cmd == 't':
        # Test sequence
        print("\n--- Test Sequence ---")
        
        print("\n[1/4] Opening for 40mm object...")
        gripper.open_for_object(40)
        time.sleep(2)
        
        print("\n[2/4] Gripping 40mm object...")
        gripper.grip_object(40)
        time.sleep(2)
        
        print("\n[3/4] Releasing...")
        gripper.release()
        time.sleep(2)
        
        print("\n[4/4] Return to default open...")
        gripper.open_for_object(50)
        
        print("\n✅ Test sequence complete!")
        
    else:
        # Try parse as width in mm
        try:
            width_mm = float(cmd)
            if GRIPPER_MIN_WIDTH_MM <= width_mm <= GRIPPER_MAX_WIDTH_MM:
                angle = gripper.mm_to_angle(width_mm)
                print(f"Setting width to {width_mm}mm ({angle}°)")
                gripper.send_command(f'G{angle}')
            else:
                print(f"⚠️ Width must be {GRIPPER_MIN_WIDTH_MM}-{GRIPPER_MAX_WIDTH_MM}mm")
        except ValueError:
            print("❓ Unknown command. Use number (mm), o, c, t, or q")
"""

# =====================================================
# 🧪 Alternative: Visual Test with Camera
# =====================================================
"""
# =================================================================
# 🧪 Visual Gripper Test (with Camera Preview)
# =================================================================
# ดูขนาดวัตถุจากกล้อง แล้วทดสอบกางหุบ

print("=" * 50)
print("🧪 Visual Gripper Test")
print("=" * 50)
print("Controls:")
print("  Click object = Show grip width")
print("  SPACE = Test grip for clicked object")
print("  O = Open for object")
print("  G = Grip object")
print("  R = Release")
print("  Q = Quit")
print("=" * 50)

test_object = None

def mouse_test_callback(event, x, y, flags, param):
    global test_object
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            ox, oy, ow, oh, cx, cy, contour = obj
            if ox <= x <= ox+ow and oy <= y <= oy+oh:
                test_object = obj
                width_mm, height_mm, grip_width_mm = detector.measure_object(obj, PIXELS_PER_MM)
                print(f"\n📦 Object selected:")
                print(f"   Size: {width_mm:.1f} x {height_mm:.1f} mm")
                print(f"   Grip width: {grip_width_mm:.1f} mm")
                print(f"   Press O=Open, G=Grip, R=Release")
                break

cap = cv2.VideoCapture(CAMERA_ID)
if cap.isOpened():
    cv2.namedWindow('Gripper Test')
    cv2.setMouseCallback('Gripper Test', mouse_test_callback)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        detected_objects = detector.detect_objects(frame, THRESHOLD_VALUE, MIN_OBJECT_AREA)
        
        for obj in detected_objects:
            x, y, w, h, cx, cy, contour = obj
            width_mm, height_mm, grip_width_mm = detector.measure_object(obj, PIXELS_PER_MM)
            
            is_selected = test_object is not None and (x, y) == (test_object[0], test_object[1])
            color = (0, 0, 255) if is_selected else (0, 255, 0)
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"Grip: {grip_width_mm:.0f}mm", (x, y-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.putText(frame, "Click=Select | O=Open | G=Grip | R=Release | Q=Quit", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        if test_object:
            _, _, grip_width = detector.measure_object(test_object, PIXELS_PER_MM)
            cv2.putText(frame, f"Selected: {grip_width:.1f}mm", 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        cv2.imshow('Gripper Test', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord('o') and test_object:
            _, _, grip_width = detector.measure_object(test_object, PIXELS_PER_MM)
            gripper.open_for_object(grip_width)
        elif key == ord('g') and test_object:
            _, _, grip_width = detector.measure_object(test_object, PIXELS_PER_MM)
            gripper.grip_object(grip_width)
        elif key == ord('r'):
            gripper.release()
    
    cap.release()
    cv2.destroyAllWindows()

🧪 Gripper Dynamic Test
Commands:
  [number] = Set gripper width (mm), e.g. '30' = 30mm
  o = Open fully (74mm)
  c = Close fully (5mm)
  t = Test sequence (open 50mm → grip 30mm → release)
  q = Quit
Opening to 74mm (22°)
Closing to 5mm (96°)
Opening to 74mm (22°)

--- Test Sequence ---

[1/4] Opening for 40mm object...
🦾 Opening for 40.0mm object → open to 55.0mm (42°)

[2/4] Gripping 40mm object...
🦾 Gripping 40.0mm object → close to 37.0mm (64°)

[3/4] Releasing...
🦾 Releasing → open to 50.0mm (47°)

[4/4] Return to default open...
🦾 Opening for 50.0mm object → open to 65.0mm (31°)

✅ Test sequence complete!
Closing to 5mm (96°)
Exit test mode
🧪 Visual Gripper Test
Controls:
  Click object = Show grip width
  SPACE = Test grip for clicked object
  O = Open for object
  G = Grip object
  R = Release
  Q = Quit


NameError: name 'detector' is not defined

In [26]:
# Connect Robot
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX)
robot.connect(ROBOT_IP)

Clearing Errors...
Enabling Robot...
✅ Robot connected!


True

In [27]:
# Create Object Detector
detector = ObjectDetector()
print("✓ ObjectDetector created")

✓ ObjectDetector created


In [28]:
# Move robot to home
print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

⚠️ Robot will move to HOME!
🤖 Moving to HOME...
✅ HOME


---
## 7️⃣ 📷 Capture Background

**⚠️ เอาวัตถุออกจากพื้นให้หมด!**

In [12]:
# Move robot to home first
print("⚠️ Robot will move to HOME!")
if input("Confirm? (y/n): ").lower() == 'y':
    robot.home()

⚠️ Robot will move to HOME!
🤖 Moving to HOME...
✅ HOME


In [30]:
# Capture Background
print("📷 Capture Background")
print("⚠️ Remove ALL objects from the workspace!")
print("="*50)

cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("Press SPACE to capture background, Q to cancel")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        cv2.putText(frame, "Remove ALL objects | SPACE = Capture | Q = Cancel", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        cv2.imshow('Capture Background', frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            print("Cancelled")
            break
        elif key == ord(' '):
            detector.set_background(frame)
            cv2.imwrite('background.jpg', frame)
            print("✅ Background saved to background.jpg")
            time.sleep(1)
            break
    
    cap.release()
    cv2.destroyAllWindows()

📷 Capture Background
⚠️ Remove ALL objects from the workspace!
Press SPACE to capture background, Q to cancel
✅ Background captured!
✅ Background saved to background.jpg


---
## 8️⃣ 📏 Calibrate PIXELS_PER_MM

ใช้วัตถุที่ทราบขนาดเพื่อ Calibrate

In [31]:
# ==========================================
# 📏 PIXELS_PER_MM Calibration (Manual Only)
# ==========================================
print("📏 Manual Calibration Mode")
print("="*50)
print("1. วางวัตถุที่รู้ขนาด (เช่น ไม้บรรทัด หรือ บัตรแข็ง)")
print("2. จัดกล้องให้ขนานกับวัตถุ")
print("3. กด SPACEBAR เพื่อหยุดภาพและลากกรอบวัด")
print("="*50)

# 1. รับค่าความกว้างจริง (mm)
try:
    known_width_mm = float(input("🔹 ใส่ความกว้างจริงของวัตถุ (mm): "))
except ValueError:
    print("❌ กรุณาใส่ตัวเลขเท่านั้น")
    raise SystemExit

# 2. เปิดกล้อง
cap = cv2.VideoCapture(CAMERA_ID)

if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("\n📸 กล้องเปิดแล้ว...")
    print("   👉 กด [SPACE] เพื่อหยุดภาพและเริ่มลากเส้น")
    print("   👉 กด [q] เพื่อยกเลิก")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("❌ Failed to grab frame")
            break
        
        display_frame = frame.copy()
        cv2.putText(display_frame, "Position object & Press SPACE", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        cv2.imshow('Manual Calibration', display_frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord(' '):
            print("\n🖱️  กรุณาลากเมาส์เป็นกรอบสี่เหลี่ยมครอบความกว้างวัตถุ...")
            print("    (ลากเสร็จแล้วกด SPACE หรือ ENTER อีกครั้งเพื่อยืนยัน)")
            
            r = cv2.selectROI('Manual Calibration', frame, fromCenter=False, showCrosshair=True)
            
            width_pixels = r[2]
            
            if width_pixels > 0:
                PIXELS_PER_MM = width_pixels / known_width_mm
                
                p1 = (int(r[0]), int(r[1]))
                p2 = (int(r[0] + r[2]), int(r[1] + r[3]))
                cv2.rectangle(frame, p1, p2, (0, 255, 0), 3)
                cv2.putText(frame, f"Width: {width_pixels}px", (p1[0], p1[1]-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                cv2.imshow('Manual Calibration', frame)
                cv2.waitKey(1000)
                
                print(f"\n✅ Calibrated Successfully!")
                print(f"   Measured Width : {width_pixels} pixels")
                print(f"   Real Width     : {known_width_mm} mm")
                print(f"   ----------------------------------")
                print(f"   🎯 PIXELS_PER_MM = {PIXELS_PER_MM:.4f}")
                print(f"   ----------------------------------")
                break
            else:
                print("⚠️ คุณไม่ได้ลากพื้นที่ (Width = 0). ลองใหม่.")

        elif key == ord('q'):
            print("❌ ยกเลิกการ Calibrate")
            break

    cap.release()
    cv2.destroyAllWindows()

📏 Manual Calibration Mode
1. วางวัตถุที่รู้ขนาด (เช่น ไม้บรรทัด หรือ บัตรแข็ง)
2. จัดกล้องให้ขนานกับวัตถุ
3. กด SPACEBAR เพื่อหยุดภาพและลากกรอบวัด

📸 กล้องเปิดแล้ว...
   👉 กด [SPACE] เพื่อหยุดภาพและเริ่มลากเส้น
   👉 กด [q] เพื่อยกเลิก

🖱️  กรุณาลากเมาส์เป็นกรอบสี่เหลี่ยมครอบความกว้างวัตถุ...
    (ลากเสร็จแล้วกด SPACE หรือ ENTER อีกครั้งเพื่อยืนยัน)

✅ Calibrated Successfully!
   Measured Width : 338 pixels
   Real Width     : 150.9 mm
   ----------------------------------
   🎯 PIXELS_PER_MM = 2.2399
   ----------------------------------


---
## 9️⃣ 🔍 Test Object Detection & Measurement

ทดสอบตรวจจับและวัดขนาดวัตถุ

In [32]:
# Test Object Detection
print("🔍 Object Detection Test")
print("Place objects and see detection + measurements")
print("Press Q to quit")

cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        objects = detector.detect_objects(frame, THRESHOLD_VALUE, MIN_OBJECT_AREA)
        
        for i, obj in enumerate(objects):
            x, y, w, h, cx, cy, contour = obj
            
            width_mm, height_mm, grip_width_mm = detector.measure_object(obj, PIXELS_PER_MM)
            grip_angle = gripper.calculate_grip_angle(grip_width_mm)
            
            color = (0, 255, 0) if i == 0 else (255, 200, 0)
            
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)
            cv2.drawContours(frame, [contour], -1, color, 1)
            
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int32(box)
            cv2.drawContours(frame, [box], 0, (255, 0, 255), 2)
            
            info_y = y - 10 if y > 80 else y + h + 20
            cv2.putText(frame, f"Object {i+1}", (x, info_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Size: {width_mm:.0f}x{height_mm:.0f}mm", (x, info_y+18),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.putText(frame, f"Grip: {grip_width_mm:.0f}mm -> {grip_angle}deg", (x, info_y+36),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.putText(frame, f"Detected: {len(objects)} objects | PIXELS_PER_MM: {PIXELS_PER_MM:.2f} | Q=Quit", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Object Detection', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

🔍 Object Detection Test
Place objects and see detection + measurements
Press Q to quit


---
## 🔟 🎯 Pick-and-Place with Dynamic Gripper

คลิกวัตถุ → Gripper เปิดพอเหมาะ → คีบตามขนาด → หยิบ

In [ ]:
# =================================================================
# 🎯 Pick-and-Place with Dynamic Gripper (v4.1)
# =================================================================

selected_object = None
detected_objects = []

def mouse_callback_pick(event, x, y, flags, param):
    global selected_object
    
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            ox, oy, ow, oh, cx, cy, contour = obj
            if ox <= x <= ox+ow and oy <= y <= oy+oh:
                selected_object = obj
                
                width_mm, height_mm, grip_width_mm = detector.measure_object(obj, PIXELS_PER_MM)
                grip_angle = gripper.calculate_grip_angle(grip_width_mm)
                robot_x, robot_y = robot.pixel_to_robot(cx, cy)
                
                # Calculate gripper widths
                open_width = min(GRIPPER_MAX_WIDTH_MM, grip_width_mm + GRIPPER_OPEN_MARGIN_MM)
                
                print(f"\n" + "="*50)
                print(f"📦 SELECTED OBJECT")
                print(f"="*50)
                print(f"   📍 Position: Pixel({cx}, {cy}) → Robot({robot_x:.1f}, {robot_y:.1f})")
                print(f"   📐 Size: {width_mm:.1f} x {height_mm:.1f} mm")
                print(f"   🦾 Grip Width (from camera): {grip_width_mm:.1f} mm")
                print(f"   📖 Open Width: {open_width:.1f} mm")
                print(f"   ⚙️ Grip Angle: {grip_angle}°")
                print(f"   📏 Z Height: {Z_FLOOR} (floor level)")
                print(f"="*50)
                print(f"   Press SPACE to pick, R to reset, Q to quit")
                break

def pick_object_with_display(obj):
    """Execute pick-and-place with dynamic gripper width from camera"""
    ox, oy, ow, oh, cx, cy, contour = obj
    
    # Measure object from camera
    width_mm, height_mm, grip_width_mm = detector.measure_object(obj, PIXELS_PER_MM)
    grip_angle = gripper.calculate_grip_angle(grip_width_mm)
    robot_x, robot_y = robot.pixel_to_robot(cx, cy)
    
    print("\n" + "="*60)
    print("🤖 PICKING OBJECT (Dynamic Gripper from Camera)")
    print("="*60)
    print(f"   Size: {width_mm:.1f} x {height_mm:.1f} mm")
    print(f"   Grip Width from Camera: {grip_width_mm:.1f} mm")
    print(f"   Position: ({robot_x:.1f}, {robot_y:.1f})")
    print("="*60)
    
    # Step 1: Open gripper พอเหมาะกับขนาดวัตถุ
    print(f"\n[1/7] Opening gripper for {grip_width_mm:.1f}mm object...")
    gripper.open_for_object(grip_width_mm)
    time.sleep(2)
    
    # Step 2: Move to safe height
    print(f"\n[2/7] Moving to ({robot_x:.1f}, {robot_y:.1f}, Z={Z_SAFE})...")
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, R_ANGLE, wait_time=3)
    
    # Step 3: Lower to floor
    print(f"\n[3/7] Lowering to Z={Z_FLOOR}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_FLOOR, R_ANGLE, wait_time=2)
    
    # Step 4: Grip ตามขนาดวัตถุจากกล้อง
    print(f"\n[4/7] Gripping {grip_width_mm:.1f}mm object...")
    gripper.grip_object(grip_width_mm)
    time.sleep(1.5)
    
    # Step 5: Lift
    print(f"\n[5/7] Lifting to Z={Z_SAFE}...")
    robot.move_to_and_wait(robot_x, robot_y, Z_SAFE, R_ANGLE, wait_time=2)
    
    # Step 6: Move to drop
    drop_x, drop_y, drop_z, drop_r = DROP_POS
    print(f"\n[6/7] Moving to drop position ({drop_x}, {drop_y}, {drop_z})...")
    robot.move_to_and_wait(drop_x, drop_y, drop_z, drop_r, wait_time=3)
    
    # Step 7: Release ปล่อยวัตถุ
    print("\n[7/7] Releasing object...")
    gripper.release()
    time.sleep(1)
    
    print("\n" + "="*60)
    print("✅ PICK-AND-PLACE COMPLETE!")
    print("="*60)

# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
if not cap.isOpened():
    print(f"❌ Cannot open camera {CAMERA_ID}")
else:
    print("="*50)
    print("🎯 Pick-and-Place Mode (Dynamic Gripper v4.1)")
    print("="*50)
    print("Click object → SPACE to pick → Q to quit")
    print("="*50)
    
    cv2.namedWindow('Pick and Place')
    cv2.setMouseCallback('Pick and Place', mouse_callback_pick)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        detected_objects = detector.detect_objects(frame, THRESHOLD_VALUE, MIN_OBJECT_AREA)
        
        for i, obj in enumerate(detected_objects):
            ox, oy, ow, oh, cx, cy, contour = obj
            
            width_mm, height_mm, grip_width_mm = detector.measure_object(obj, PIXELS_PER_MM)
            grip_angle = gripper.calculate_grip_angle(grip_width_mm)
            open_width = min(GRIPPER_MAX_WIDTH_MM, grip_width_mm + GRIPPER_OPEN_MARGIN_MM)
            
            is_selected = selected_object is not None and (ox, oy) == (selected_object[0], selected_object[1])
            color = (0, 0, 255) if is_selected else (0, 255, 0)
            thickness = 3 if is_selected else 2
            
            cv2.rectangle(frame, (ox, oy), (ox+ow, oy+oh), color, thickness)
            cv2.circle(frame, (cx, cy), 6, (0, 0, 255), -1)
            
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)
            box = np.int32(box)
            cv2.drawContours(frame, [box], 0, (255, 0, 255), 2)
            
            info_y = max(60, oy - 50)
            cv2.rectangle(frame, (ox, info_y-45), (ox+200, info_y+5), (0, 0, 0), -1)
            cv2.putText(frame, f"Size: {width_mm:.0f}x{height_mm:.0f}mm", (ox+5, info_y-25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255, 255, 255), 1)
            cv2.putText(frame, f"Grip: {grip_width_mm:.0f}mm Open: {open_width:.0f}mm", (ox+5, info_y-5),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 255), 1)
        
        cv2.rectangle(frame, (0, 0), (640, 40), (50, 50, 50), -1)
        cv2.putText(frame, f"Objects: {len(detected_objects)} | Dynamic Gripper v4.1 | SPACE=Pick | Q=Quit", 
                   (10, 28), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (255, 255, 255), 1)
        
        if selected_object:
            cv2.putText(frame, "[SELECTED - Press SPACE to pick]", 
                       (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        
        cv2.imshow('Pick and Place', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord('r'):
            selected_object = None
            print("\n🔄 Reset")
        elif key == ord(' ') and selected_object:
            pick_object_with_display(selected_object)
            selected_object = None
        elif key == ord('h'):
            robot.home()
    
    cap.release()
    cv2.destroyAllWindows()

🎯 Pick-and-Place Mode (Dynamic Gripper v4.1)
Click object → SPACE to pick → Q to quit
🤖 Moving to HOME...
✅ HOME


In [24]:
# Home robot
if input("Move to HOME? (y/n): ").lower() == 'y':
    robot.home()

🤖 Moving to HOME...
Command Error: [WinError 10054] An existing connection was forcibly closed by the remote host
✅ HOME


---
## 1️⃣1️⃣ Cleanup

In [ ]:
# Disconnect
gripper.disconnect()